# BLU03 - Exercises Notebook

In [1]:
import hashlib # for grading purposes
import math
import numpy as np
import pandas as pd
import requests
import sqlalchemy

from bs4 import BeautifulSoup

## Part A - SQL exercises

### Querying a StockDatabase with a SQL client

Open your favorite SQL client and connect to the StockDatabase.
The connection settings are the following.

* host: batch4-s02-db-instance.ctq2kxc7kx1i.eu-west-1.rds.amazonaws.com
* port: 5432
* user: ldsa_student
* database: s02_db
* schema: public
* password: XXX (shared through slack)

This is a different schema than the one we used in the learning notebooks (don't forget to change to this schema, see the Learning Notebook). This schema contains information about stock tickers, including their location, some financial information, and whether they are in certain indices.

The tables in this schema are the following:

1. Stock: has information on ticker, stock name, and sector and industry information.
2. Financial: contains latest price and marketcap of all tickers.
3. Location: contains information about where the company is located.
4. Info: contains information about if a given company is in a certain index (SP500 for example).

You can preview these tables using the SQL client.

## Note:

Since some of the table and column names are reserved names, so they have to be written in double quotes in
the queries.  For example in the Financial table (as financial), you can call the Name column using
`financial."Name"`

### Q1. What is the Ticker of the company with name "ExlService Holdings, Inc."

Write a query that selects the ticker of the company "ExlService Holdings, Inc.", and run it in the SQL client.

Then, assign the result to variable q1_answer (just copy and paste the name you obtained).

In [2]:
password = 'R4Fr4P3aAgMYBqqP'

In [3]:
# YOUR CODE HERE
username = 'ldsa_student'
password = 'R4Fr4P3aAgMYBqqP'  # the password is not XXX by the way
host_name = 'batch4-s02-db-instance.ctq2kxc7kx1i.eu-west-1.rds.amazonaws.com'
port = 5432
db_name = 's02_db'
schema = 'public'

conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host_name, port, db_name)
conn_args = {'options': '-csearch_path={}'.format(schema)}

In [4]:
# Create the connection string
conn_str = f'postgresql://{username}:{password}@{host_name}:{port}/{db_name}'
conn_args = {'options': f'-csearch_path={schema}'}

# Create the engine
engine = sqlalchemy.create_engine(conn_str, connect_args=conn_args)

# Query
query = 'SELECT "Ticker" FROM stock WHERE "Name" = \'ExlService Holdings, Inc.\';'

# Execute the query and fetch the result
q1_answer_df = pd.read_sql_query(query, engine)

# Extract the ticker value
q1_answer = q1_answer_df['Ticker'].iloc[0] if not q1_answer_df.empty else "No result found"

q1_answer

'EXLS'

In [5]:
expected_hash = '404fd7677ca78c286e29c4156fdcf26468198af98e94bbb7ce9686e674f4c0ec'
assert hashlib.sha256(q1_answer.encode()).hexdigest() == expected_hash

### Q2. Count how many companies hare located in State 'CA'

Write a query that counts the number companies that are located in state (`State`) 'CA'

Then, assign the result to variable q2_answer (just copy and paste the value).

In [6]:
# YOUR CODE HERE
query = 'SELECT COUNT(*) AS company_count FROM location WHERE "State" = \'CA\';'

# Execute the query and fetch the result
q2_answer_df = pd.read_sql_query(query, engine)

# Extract the count value
q2_answer = q2_answer_df['company_count'].iloc[0]

q2_answer

531

In [7]:
expected_hash = '891d46993a36d78392247c642138cede01d9841daab1d945709755b5194597c4'
assert hashlib.sha256(str(q2_answer).encode()).hexdigest() == expected_hash

### Q3. Find the name of the 3 texas financial sector companies that have the highest lastprice.

Find the names of the 3 financial sector (`Sector`) companies that have the highest lastprice (`Lastprice`) and are located in state (`State`) 'TX'

That's quite a lot to ask!

Let's break it down. Write a query that:

* Finds the companies that are located in state 'TX'
* Filters only based on the 'Financial' sector
* Sorts by lastprice and gets the 3 highest

Hint: Be carefull with the NULL values in `Lastprice`

Then, create a list with the results ordered and assign it to variable q3_answer.


In [8]:
# YOUR CODE HERE


#  SQL query
query = """
SELECT s."Name"
FROM stock s
JOIN location l ON s."Name" = l."Name"
JOIN financial f ON s."Name" = f."Name"
WHERE l."State" = 'TX'
AND s."Sector" = 'Financial'
AND f."Lastprice" IS NOT NULL
ORDER BY f."Lastprice" DESC
LIMIT 3;
"""

# Execute the query and fetch the results
q3_answer_df = pd.read_sql_query(query, engine)

# Extract the names into a list
q3_answer = q3_answer_df['Name'].tolist()


In [9]:
expected_hash = '1467aec780d9e25a7a2b49d528bfe51ed9a166ae3b5362e58f2b59cb85ecc7c8'
assert hashlib.sha256(str(q3_answer).encode()).hexdigest() == expected_hash

### Q4. Find out what industries have companies that have both stocks in th the Nasdaq100 and a total marketcap below 50.

Write a query to retrieve the names of industries that have companies that meet both the following criteria:
- The companies have stocks listed in the Nasdaq100.
- The total marketcap of those companies is below 50.


Order the results by the total marketcap in ascending order (meaning the industry that contains the companies with the lowest total marketcap first). Create a list with the results, and assign it to variable q4_answer.

In [10]:

# SQL query
query = """
SELECT s."Industry"
FROM "stock" s
JOIN "financial" f ON s."Name" = f."Name"
JOIN "info" i ON s."Name" = i."Name"
WHERE i."Nasdaq100" = '1'
GROUP BY s."Industry"
HAVING SUM(f."Marketcap") < 50
ORDER BY SUM(f."Marketcap");
"""

# Execute the query and fetch the results
q4_answer_df = pd.read_sql_query(query, engine)

# Extract the industry names into a list
q4_answer = q4_answer_df['Industry'].tolist()



In [11]:
expected_hash = '73c5fee314b6a6dfcb6392933fd5174152f172e73e66ae0a83b713c26c325a36'
assert hashlib.sha256(str(q4_answer).encode()).hexdigest() == expected_hash

### Q5. Find out what sector had the highest average stock price (`lastprice`)

Find out which state has the healthcare companies with the highest average marketcap.

Assign this state to variable q5_answer_1.

Also find out the max lastprice of the healthcare companies on that state and assign the result to q5_answer_2


In [12]:
# YOUR CODE HERE
query = '''
SELECT l."State", AVG(f."Marketcap") as avg_marketcap
FROM "location" l
JOIN "stock" s ON l."Name" = s."Name"
JOIN "financial" f ON s."Name" = f."Name"
WHERE s."Sector" = 'Healthcare'
GROUP BY l."State"
ORDER BY avg_marketcap DESC
LIMIT 1;
'''
q5_answer_1= pd.read_sql_query(query, engine)
q5_answer_1= q5_answer_1['State'][0]

query = '''
SELECT MAX(f."Lastprice") as max_last_price
FROM "location" l
JOIN "stock" s ON l."Name" = s."Name"
JOIN "financial" f ON s."Name" = f."Name"
WHERE s."Sector" = 'Healthcare'
AND l."State" = 'RI';
'''
q5_answer_2= pd.read_sql_query(query, engine)['max_last_price'][0]



q5_answer_1, q5_answer_2

('RI', 102.15)

In [13]:
expected_sector_hash = '91d19006c4397461ab9912a25f8d7b52ad39d438d884005edd54006f576d7869'
assert hashlib.sha256(q5_answer_1.encode()).hexdigest() == expected_sector_hash, "Wrong sector!"

expected_max_price_hash = '420608fcbb711208c6c09c388ea97562c285fc86359069e79b8bb583bd2aacf6'
assert hashlib.sha256(str(q5_answer_2).encode()).hexdigest() == expected_max_price_hash, "Wrong max price!"

## Part B - Public APIs


-----------------------------------

In this exercises, the goal is to get data from a public API. We'll go full geek, and use a Pokemon API hosted by the LDSA for this BLU! (credit for the data goes to user `fanzeyi` on Github)

The base URL of the API is the following: https://pokemon-api.lisbondatascience.org/

In order to complete the exercises, you'll have to navigate to the API's documentation (`ui` endpoint) on your browser. More specifically, you'll have to learn what are the different endpoints from which you can GET information.

<br>

<img src="media/api-image.jpg" width=600>

<br>

### Q6. Find all of Pikachu's evolutions!

As you might know, Pokemon evolve as they grow. Several Pokemon keep a similar name when they evolve. Let's consider the most famous Pokemon, Pikachu:

<br>

<img src="media/pikachu.png" width=300>

<br>

Use the API to find all Pikachu's evolutions! The names of Pikachu evolutions all end in `chu`, so you need to get the pokemons whose names contain that substring. However, you'll also have to filter for "Electric" type Pokemon, since there are a couple of results unrelated to Pikachu.

Extract their ids from the `["id"]` attribute of each result, in the order they are returned, and assign the resulting list to the `q6_answer_ids` variable.
Also extract their attack scores (`["base"]["Attack"]`) and assign them to variable `q6_answer_attack`

In [14]:
base_url = "https://pokemon-api.lisbondatascience.org/"

In [15]:
# Do an HTTP GET request to the Pokemon API to get information about 
# all Pokemons with "chu" in their name
# response = ...
# q6_answer_names = ...
# q6_answer_speeds = ...

# YOUR CODE HERE


# Base URL of the Pokemon API
base_url = "https://pokemon-api.lisbondatascience.org/"
endpoint = 'pokemon'

# Parameters for the GET request
params = {
    'name': 'chu',
    'exact': False,
    'poke_type': 'Electric'
}

# Execute the GET request
response = requests.get(f'{base_url}{endpoint}', params=params)

# Check if the request was successful
if response.status_code == 200:
    # Extracting Pokemon IDs and Attack scores from the response
    pokemons = response.json()
    q6_answer_ids = [pokemon['id'] for pokemon in pokemons if 'chu' in pokemon['name']['english'].lower()]
    q6_answer_attacks = [pokemon['base']['Attack'] for pokemon in pokemons if 'chu' in pokemon['name']['english'].lower()]
else:
    q6_answer_ids = []
    q6_answer_attacks = []

q6_answer_ids, q6_answer_attacks


([25, 26, 172], [55, 90, 40])

In [16]:
assert type(q6_answer_ids) == list, "Ids must be in a list"
assert type(q6_answer_attacks) == list, "Attacks must be in a list"

expected_ids_hash = 'a160dccf2a5c35ac2760e9846997d5898bd6474af6fe776da487c5ed6b2961e5'
assert hashlib.sha256(str(q6_answer_ids).encode()).hexdigest() == expected_ids_hash, "Wrong names!"

expected_attacks_hash = '4ab8649e9e835ddbcafe15be29b95ff7994be34434a366ac731add7a53b12921'
assert hashlib.sha256(str(q6_answer_attacks).encode()).hexdigest() == expected_attacks_hash, "Wrong speeds!"

### Q7 Find the the strongest and most accurate Pokemon move!

Now, use a different endpoint to find out which Pokemon moves have an `accuracy` higher than 95  and `power` stat of 200 or higher.

Extract their `enames` (english names) and `types` and assign the resulting lists to variables `q7_answer_names` and `q7_answer_types` respectively.

In [17]:
# Do an HTTP GET request to find which Pokemon moves have 200 or more power.

# YOUR CODE HERE


# Base URL of the Pokemon API
base_url = "https://pokemon-api.lisbondatascience.org/"
endpoint = 'moves'

# Parameters for the GET request
params = {
    'accuracy': 95,
    'power': 200,
    'power_operator': 'gte',
    'accuracy_operator': 'gt'
}

# Execute the GET request
response = requests.get(f'{base_url}{endpoint}', params=params)

# If the request was successful, proceed to extract the information
if response.status_code == 200:
    moves = response.json()
    q7_answer_names = [move['ename'] for move in moves]
    q7_answer_types = [move['type'] for move in moves]
else:
    print("Failed to retrieve data:", response.status_code)

# Print the result or process it as needed
print(q7_answer_names)
print(q7_answer_types)


['Self-Destruct', 'Explosion']
['Normal', 'Normal']


In [18]:
assert type(q7_answer_names) == list, "Moves names must be in a list!"
assert len(q7_answer_names) == 2, "Wrong number of moves!"

expected_moves_hash = 'a7b4c8bc5e6e205ab29e8255537e3bb8ae04269b5b329a3e7c5984ff45542df1'
assert hashlib.sha256(str(q7_answer_names).encode()).hexdigest() == expected_moves_hash

assert type(q7_answer_types) == list, "Moves types must be in a list!"
assert len(q7_answer_types) == 2, "Wrong number of moves!"

expected_types_hash = '45a4bf1d2f50d7a4844324b44de10a89b916280b1c42475b429c587f825802a6'
assert hashlib.sha256(str(q7_answer_types).encode()).hexdigest() == expected_types_hash

## Part C - Web scraping

In this exercise, we're going to use web scraping to get data from the page of a former LDSA student, Bork Pawson!
Bork has kindly made his very simple and amateurish website available for us to scrape!

You can find his website here: https://s02-infrastructure.s3.eu-west-1.amazonaws.com/ldsa-bork/index.html

### Q8. Scrape Bork's AWESOME honourable mentions

Bork has written 3 things that didn't fit in the webpage. You can find them listed on the top of the images.
Scrape the 3 items in order, using the `requests` and `BeautifulSoup` library, store them in a list, and assign it to the `q10_answer` variable. No cheating! 

In [19]:
# Assign the URL of the page to be scraped to variable url
# url = ...
# YOUR CODE HERE
url = "https://s02-infrastructure.s3.eu-west-1.amazonaws.com/ldsa-bork/index.html"

# Do a GET request to get the page content, using the url we've just defined
# response = ...
# YOUR CODE HERE
response = requests.get(url)

# Instantiate a soup object using the response of the GET request
# YOUR CODE HERE
# Check if the request was successful
if response.status_code == 200:
    # Instantiate a soup object using the response of the GET request
    soup = BeautifulSoup(response.content, 'html.parser')
    
# Now it's the tricky part!
# Parse the soup in order to retrieve the list of things.
# In the end, store the favourite things in a list and assign it to variable q10_answer.
# YOUR CODE HERE
# Find the unordered list by class name 'honourable-mentions'
honourable_mentions_list = soup.find('ul', class_='honourable-mentions')

# Extract the text of each list item within the unordered list
q8_answer = [item.text for item in honourable_mentions_list.find_all('li')]

# q8_answer now contains the 3 honourable mentions
print(q8_answer)

['puddles', 'sniffing butts', 'dog park']


In [20]:
expected_hash = 'ae34341c0912945834a9d489f0b0b1a3d42117e00521db4abbdbdda15e0ce6c3'
assert hashlib.sha256(str(sorted(q8_answer)).encode()).hexdigest() == expected_hash

### Q9. Find the tennis ball tag

Scrape the tag containing the tennis ball image that is on the center of the grid with Bork's favourite things.
Assign the tag (not the image content) to variable `q9_answer`.

Note: You'll have to find a different way to pass the attribute you want to filter, since the attribute name conflicts with an argument of the `find` function. You can figure out how to do this in the [BeautifulSoup documentation](https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=find#the-keyword-arguments)!

In [21]:
# Assign the URL of the page to be scraped to variable url
# url = ...
# YOUR CODE HERE
url = "https://s02-infrastructure.s3.eu-west-1.amazonaws.com/ldsa-bork/index.html"

# Do a GET request to get the page content, using the url we've just defined
# response = ...
# YOUR CODE HERE
response = requests.get(url)

# Instanciate a soup object using the response of the GET request
# YOUR CODE HERE
if response.status_code == 200:
    # Instantiate a soup object using the response of the GET request
    soup = BeautifulSoup(response.content, 'html.parser')

# Parse the soup in order to retrieve the tag of the tennis ball image.
# Assign it to variable q11_answer.
# YOUR CODE HERE
q9_answer = soup.find('img', attrs={'name': 'ball'})

In [22]:
expected_hash = '369917cf8ea4d7906841cb6e6c264b124911e6d805bd122a23ffcee8fcb67de7'
assert hashlib.sha256(str(q9_answer).encode()).hexdigest() == expected_hash